In [1]:
import kagglehub
import pandas as pd
import numpy as np
import os
import getpass
from openai import OpenAI
import scipy
from langchain_openai import ChatOpenAI
from typing import List, Any, Optional, Dict
from typing_extensions import TypedDict
import re
from pprint import pprint
from pydantic import BaseModel, Field, field_validator
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.schema import HumanMessage, SystemMessage
import time
from pathlib import Path
from langchain_community.callbacks import get_openai_callback
import re
import matplotlib.pyplot as plt
import io
import contextlib
import base64
import plotly
from category_encoders import *
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import psutil
import warnings
import seaborn as sns
import sys
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
from dataclasses import dataclass, field

# Import iterative system components
from core.agents import generate_iterative_personas, ITERATIVE_AGENT_ROLES
from orchestrators.orchestrator import IterativeOrchestrator
from reporting.exporters import IterativeReportExporter
from reporting.task_boards import TaskChecklist
from reporting.summarizer import ReportSummarizer
from core.pipeline_state import PipelineState
from reporting.QA import QualityAssurance
from reporting.validator import unit_test_report

/Users/suchitranayak/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")  # Make sure this is set in your environment

if not api_key:
    print("⚠️ Warning: OPENAI_API_KEY not found. Please set your API key.")
  
# Initialize LLM (same as your original setup)
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    #model="gpt-4.1-mini",   # Use gpt-4o if you have access #nano
    temperature=0.5,
    openai_api_key=api_key
)

llm_coder = ChatOpenAI(
    #model="gpt-4.1-nano",
    model="gpt-4.1-mini",   # Use gpt-4o if you have access #nano
    temperature=0.2,
    openai_api_key=api_key
)

print("✅ LLM configured successfully!")

✅ LLM configured successfully!


In [3]:
# Option 2: Load your own CSV
df = pd.read_excel('data/Pumpkin_Seeds_Dataset.xlsx')
#df_test = pd.read_csv('data/df_test.csv')

print(f"✅ Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"📋 Columns: {list(df.columns)}")


✅ Dataset loaded: 2500 rows, 13 columns
📋 Columns: ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Convex_Area', 'Equiv_Diameter', 'Eccentricity', 'Solidity', 'Extent', 'Roundness', 'Aspect_Ration', 'Compactness', 'Class']


In [4]:
# Generate context
summary_stats = df.describe(include='all').to_string()
column_info = df.dtypes.to_string()
col_names = ", ".join(df.columns)
context = f"""## Dataset: Pumpkin Seed Data

### Schema:
{column_info}

### Summary Statistics:
{summary_stats}

### Column Names:
{col_names}"""

In [5]:
# Project configuration
PROJECT_NAME = "pumpkin_seeds_iterative_v4"
PIPELINE_TASKS = [
    "Exploratory Data Analysis (EDA)",
    "Feature Engineering", 
    "Model Selection & Evaluation"
]

In [6]:
# Initialize Pipeline State
pipeline_state = PipelineState(project_name=PROJECT_NAME)

# Check existing project state
completed_phases = pipeline_state.get_completed_phases()
print(f"\n📁 Iterative Project: {PROJECT_NAME}")

if completed_phases:
    print(f"✅ Found existing phases: {completed_phases}")
else:
    print("🆕 Starting new iterative project")

# Initialize defaults
remaining_tasks = PIPELINE_TASKS
current_df = df

# Auto-resume from last completed phase
if completed_phases:
    last_phase = completed_phases[-1]
    print(f"🔄 Auto-resuming from last completed phase: {last_phase}")
    pipeline_state.load_from_phase(last_phase)
    
    try:
        last_index = PIPELINE_TASKS.index(last_phase) + 1
        remaining_tasks = PIPELINE_TASKS[last_index:]
        current_df = pipeline_state.df if pipeline_state.df is not None else df
        print(f"📊 Loaded dataframe shape: {current_df.shape}")
    except (ValueError, IndexError):
        print("❌ Error in resume logic, starting fresh")
        remaining_tasks = PIPELINE_TASKS
else:
    print("🆕 Starting fresh iterative pipeline...")
    remaining_tasks = PIPELINE_TASKS

# Execution Summary
print(f"\n📋 Iterative Execution Plan:")
print(f"   Project: {PROJECT_NAME}")
print(f"   Architecture: 3-Agent Iterative (Planner → Developer → Auditor → Developer)")
print(f"   Completed phases: {completed_phases}")
print(f"   Remaining phases: {remaining_tasks}")
print(f"   DataFrame shape: {current_df.shape}")

if not remaining_tasks:
    print("\n✅ All phases completed! Nothing to do.")
    print("💡 Tip: Change PROJECT_NAME or delete cache to start over")
    exit()

print(f"\n🚀 Will execute {len(remaining_tasks)} phase(s) using iterative process")



📁 Iterative Project: pumpkin_seeds_iterative_v4
🆕 Starting new iterative project
🆕 Starting fresh iterative pipeline...

📋 Iterative Execution Plan:
   Project: pumpkin_seeds_iterative_v4
   Architecture: 3-Agent Iterative (Planner → Developer → Auditor → Developer)
   Completed phases: []
   Remaining phases: ['Exploratory Data Analysis (EDA)', 'Feature Engineering', 'Model Selection & Evaluation']
   DataFrame shape: (2500, 13)

🚀 Will execute 3 phase(s) using iterative process


In [7]:
# Main execution loop
for task in remaining_tasks:
    print(f"\n" + "="*60)
    print(f"🔄 Running iterative pipeline phase: {task}")
    print("="*60)

    pipeline_state.clear_validation_log()

    # Timer start
    start = time.perf_counter()
    print("⏱️ Timer started")

    # Token tracking
    if get_openai_callback is not None:
        ctx = get_openai_callback()
    else:
        ctx = None

    with ctx as cb:
        # Generate 3-agent personas for iterative workflow
        print(f"\n👥 Generating iterative team personas for {task}...")
        personas = generate_iterative_personas(context=context, phase=task, llm=llm)
        
        print(f"✅ Generated {len(personas.agents)} agents:")
        for agent in personas.agents:
            print(f"   - {agent.name} ({agent.role})")

        # Get summary of previous phases
        prior_summary = pipeline_state.get_contextual_summary(last_n=1)
        if prior_summary:
            print(f"\n📄 Prior context loaded: {len(prior_summary)} characters")

        # Create and run iterative orchestrator
        print(f"\n🎯 Starting iterative orchestration...")
        orchestrator = IterativeOrchestrator(
            df=current_df,
            topic=task,
            agents=personas.agents,
            llm=llm,
            llm_coder=llm_coder,
            summary=prior_summary,
            pipeline_state=pipeline_state
        )
        
        # Execute the 4-step iterative process
        phase_results = orchestrator.run()

        # Update DataFrames from orchestrator
        pipeline_state.df = orchestrator.executor.df

        # Generate summary using existing summarizer
        summary_path = pipeline_state.save_dir / "reports" / f"{PROJECT_NAME}_{task.lower().replace(' ', '_').replace('&', 'and')}_summary.html"
        summarizer = ReportSummarizer(
            results=phase_results,
            task=task,
            output_path=summary_path,
            llm=llm_coder,
            pipeline_state=pipeline_state
        )
        summarizer.run()
    
        # Save phase to pipeline cache
        pipeline_state.save_phase(task, [{k: v for k, v in r.items() if k != "images"} for r in phase_results], personas.agents)

        # Generate iterative reports
        print(f"\n📄 Generating iterative reports for {task}...")
        
        iterative_exporter = IterativeReportExporter(task=task, results=phase_results)
        iterative_exporter.export_all(project_name=PROJECT_NAME, save_dir=pipeline_state.save_dir / "reports")
        
        # Validation reports
        validation_log = pipeline_state.get_validation_log()
        if validation_log:
            validation_filename = pipeline_state.save_dir / "reports" / f"{PROJECT_NAME}_{task.lower().replace(' ', '_').replace('&', 'and')}_validation.html"
            unit_test_report(
                validation_log=validation_log,
                phase_name=task,
                project_name=PROJECT_NAME,
                filename=validation_filename
            )
        
        # QA reports
        qa_agent = QualityAssurance(pipeline_state, llm=llm_coder)
        qa_agent.validate_results(phase_results, task)
        qa_agent.export_report(phase_name=task, fmt="csv", save_dir=pipeline_state.save_dir / "reports")

        # Checklist validation
        checklist_validator = TaskChecklist(llm=llm_coder, task=task)
        report = checklist_validator.validate_phase(task, phase_results)
        checklist_validator.export_report(report, pipeline_state.save_dir / "reports" / f"{task.lower().replace(' ', '_')}_checklist_report.json")

        # Timer end
        elapsed_sec = time.perf_counter() - start
        print("\n\n")
        print(f"⏱️  {task} took {elapsed_sec:.2f}s")
        print(f"🪙  Tokens — total: {cb.total_tokens}, prompt: {cb.prompt_tokens}, completion: {cb.completion_tokens}")

        # Save metrics
        metrics_file = pipeline_state.save_dir / "reports" / f"{task}_iterative_metrics.txt"
        with open(metrics_file, "w") as f:
            f.write(f"Iterative Process: {task} took {elapsed_sec:.2f}s\n")
            f.write(f"Tokens — total: {cb.total_tokens}, prompt: {cb.prompt_tokens}, completion: {cb.completion_tokens}\n")
            if hasattr(cb, "total_cost"):
                f.write(f"Estimated API cost: ${cb.total_cost:.4f}\n")
            f.write(f"Architecture: 3-Agent Iterative (Planner → Developer → Auditor → Developer)\n")
            f.write(f"Agents: {[agent.name for agent in personas.agents]}\n")

print("\n" + "="*60)
print("✅ ITERATIVE PIPELINE EXECUTION COMPLETED!")
print("="*60)

print(f"\n📊 Final Iterative Pipeline Summary:")
summary = pipeline_state.get_project_summary()
for key, value in summary.items():
    print(f"   {key}: {value}")

print(f"\n🔄 Process Architecture: 3-Agent Iterative")
print(f"   1. Planner: Strategic planning and task decomposition")
print(f"   2. Developer: Initial implementation")
print(f"   3. Auditor: Quality review and feedback")
print(f"   4. Developer: Refined implementation based on feedback")

print(f"\n📁 Generated Files (Project: {PROJECT_NAME}):")
import glob
report_dir = pipeline_state.save_dir / "reports"
if report_dir.exists():
    for file_path in report_dir.glob("*"):
        if file_path.is_file():
            print(f"   📄 {file_path.name}")

print(f"\n💡 To view results:")
print(f"   📂 Check: {pipeline_state.save_dir}")
print(f"   📊 Reports: {report_dir}")
print(f"   🔄 Architecture: Iterative 3-Agent System")


🔄 Running iterative pipeline phase: Exploratory Data Analysis (EDA)
⏱️ Timer started

👥 Generating iterative team personas for Exploratory Data Analysis (EDA)...
✅ Generated 3 agents:
   - Alice (Planner)
   - Bob (Developer)
   - Carol (Auditor)

🎯 Starting iterative orchestration...
🔒 Data Integrity Validator initialized:
   Expected shape: (2500, 13)
   Essential columns: 13
   Target column: Class

🎯 Iterative Task: Exploratory Data Analysis (EDA)
🧩 Decomposing into subtasks...
📝 Subtasks Planned:

1. Generate descriptive statistics and distribution plots for each numerical feature to understand their ranges, central tendencies, and variability
2. Visualize the distribution of the target variable 'Class' to assess class imbalance and distribution
3. Create pairwise scatter plots and correlation heatmaps for numerical features to identify relationships, multicollinearity, and potential feature interactions
4. Analyze feature distributions and relationships across different classes 

2025-08-15 19:18:53,911 - INFO - Initial DataFrame shape: (2500, 26)
2025-08-15 19:18:53,919 - INFO - Unique classes found in 'Class': ['Çerçevelik' 'Ürgüp Sivrisi']
2025-08-15 19:18:53,926 - INFO - All class labels successfully mapped to binary.
2025-08-15 19:18:53,929 - INFO - Value counts for 'Class_binary':
Class_binary
1    1300
0    1200
Name: count, dtype: int64
2025-08-15 19:18:53,932 - INFO - Numerical features to analyze for outliers: ['Area', 'Aspect_Ration', 'Compactness', 'Convex_Area', 'Eccentricity', 'Equiv_Diameter', 'Extent', 'Major_Axis_Length', 'Minor_Axis_Length', 'Perimeter', 'Roundness', 'Solidity']
2025-08-15 19:18:53,935 - INFO - Feature 'Area': Outliers flagged = 18


{'agent': 'Bob', 'role': 'Developer (Refined)', 'final_implementation': '```python\n# Necessary imports\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.feature_selection import f_classif, SelectKBest\nfrom statsmodels.stats.outliers_influence import variance_inflation_factor\nimport logging\nimport os\n\n# Setup logging for better auditability and structured output\nlogging.basicConfig(\n    level=logging.INFO,\n    format=\'%(asctime)s - %(levelname)s - %(message)s\',\n    handlers=[\n        logging.StreamHandler()\n    ]\n)\nlogger = logging.getLogger()\n\n# Ensure output directories exist for saving visualizations and reports\nos.makedirs("visualizations", exist_ok=True)\nos.makedirs("reports", exist_ok=True)\n\nlogger.info(f"Initial DataFrame shape: {df.shape}")\n\n# --- Helper functions for modularity and clarity ---\n\ndef encode_target(df, target_col=\'Class\'):\n   

2025-08-15 19:18:54,177 - INFO - Feature 'Aspect_Ration': Outliers flagged = 11
2025-08-15 19:18:54,289 - INFO - Feature 'Compactness': Outliers flagged = 2
2025-08-15 19:18:54,390 - INFO - Feature 'Convex_Area': Outliers flagged = 17
2025-08-15 19:18:54,481 - INFO - Feature 'Eccentricity': Outliers flagged = 18
2025-08-15 19:18:54,563 - INFO - Feature 'Equiv_Diameter': Outliers flagged = 13
2025-08-15 19:18:54,683 - INFO - Feature 'Extent': Outliers flagged = 46
2025-08-15 19:18:54,777 - INFO - Feature 'Major_Axis_Length': Outliers flagged = 21
2025-08-15 19:18:54,867 - INFO - Feature 'Minor_Axis_Length': Outliers flagged = 30
2025-08-15 19:18:54,957 - INFO - Feature 'Perimeter': Outliers flagged = 16
2025-08-15 19:18:55,042 - INFO - Feature 'Roundness': Outliers flagged = 5
2025-08-15 19:18:55,138 - INFO - Feature 'Solidity': Outliers flagged = 103
2025-08-15 19:18:55,230 - INFO - Outlier summary saved to reports/outlier_summary.csv
2025-08-15 19:18:55,233 - INFO - Pearson correlatio

📄 Extracted Code:
 # Necessary imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif, SelectKBest
from statsmodels.stats.outliers_influence import variance_inflation_factor
import logging
import os

# Setup logging for better auditability and structured output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Ensure output directories exist for saving visualizations and reports
os.makedirs("visualizations", exist_ok=True)
os.makedirs("reports", exist_ok=True)

logger.info(f"Initial DataFrame shape: {df.shape}")

# --- Helper functions for modularity and clarity ---

def encode_target(df, target_col='Class'):
    """
    Encode the target variable into binary form with explicit verification.
    Returns mappin

...

2025-08-15 19:21:17,036 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'agent': 'MsClaraMartinez', 'role': 'Auditor', 'audit_feedback': "Thank you for providing the comprehensive implementation plan and code. Here's a detailed audit based on your provided workflow, execution, and results:\n\n---\n\n### **1. Model Selection & Implementation**\n\n- **Model Choice:** The use of `RandomForestClassifier` is appropriate given its robustness, interpretability, and ability to handle mixed feature types. Justification aligns with best practices.\n- **Hyperparameter Tuning:** The grid search over multiple parameters (n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features) is thorough and suitable for optimizing model performance.\n- **Reproducibility:** Fixed `random_state=42` and stratified splits ensure reproducibility and balanced class distribution across splits.\n- **Pipeline & Scaling:** Scaling features with high correlation magnitude (>0.1) is correctly executed, fitting only on training data and transforming validation and test sets ac

2025-08-15 19:22:14,390 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'agent': 'MrBrianLee', 'role': 'Developer (Refined)', 'final_implementation': '```python\nimport os\nimport logging\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport joblib\n\nfrom sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import (\n    accuracy_score,\n    precision_score,\n    recall_score,\n    f1_score,\n    confusion_matrix,\n    roc_curve,\n    roc_auc_score,\n    classification_report,\n)\nfrom sklearn.exceptions import NotFittedError\n\n# ---------------------------\n# Configuration & Setup\n# ---------------------------\n\nRANDOM_SEED = 42\n\n# Directory paths for saving artifacts\nBASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), \'..\', \'output\'))\nMODEL_DIR = os.path.join(BASE_DIR, \'models\')\nVIZ_DIR = os.path.join(BASE_DIR, \'visualizations\')\

KeyboardInterrupt: 